In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader , Dataset
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
class BMI_Dataset(Dataset):
  def __init__(self, csv_path):
    super().__init__()
    self.df = pd.read_csv(csv_path)
    self.df['Gender'] = self.df['Gender'].replace({'Male': 1, 'Female': 0})
    self.data = self.df.to_numpy() # Use self.df here

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, idx):
    features = torch.tensor(self.data[idx, :-1]).float() # Convert to float
    labels = torch.tensor(self.data[idx,-1]).long() # Convert to long for classification
    return features , labels

In [10]:
dataset = BMI_Dataset('Dataset/bmi.csv')
dataloader_train = DataLoader(dataset, batch_size=16, shuffle=True)
dataloader_test = DataLoader(dataset, batch_size=16, shuffle=False)

features, labels = next(iter(dataloader_train))
print(f"Features: {features}")
print(f"Labels: {labels}")

Features: tensor([[  1., 161., 155.],
        [  0., 163., 159.],
        [  1., 148., 141.],
        [  1., 173., 139.],
        [  1., 169., 136.],
        [  1., 188., 100.],
        [  0., 167.,  85.],
        [  1., 144.,  88.],
        [  0., 164., 160.],
        [  1., 196., 116.],
        [  1., 141.,  86.],
        [  1., 159., 104.],
        [  0., 161., 154.],
        [  0., 195., 147.],
        [  1., 177., 117.],
        [  1., 140.,  79.]])
Labels: tensor([5, 5, 5, 5, 5, 3, 4, 5, 5, 4, 5, 5, 5, 4, 4, 5])


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1808\1442832286.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['Gender'] = self.df['Gender'].replace({'Male': 1, 'Female': 0})


In [11]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(3,9)
    self.fc2 = nn.Linear(9,12)
    self.fc3 = nn.Linear(12,15)
    self.fc4 = nn.Linear(15,9)
    self.fc5 = nn.Linear(9,6) # Changed output features to 6
    # Removed self.fc6 = nn.Linear(3,1)
    # Removed self.softmax = nn.Softmax(dim=1)
  def forward(self,x):
    x = F.elu(self.fc1(x))
    x = F.elu(self.fc2(x))
    x = F.elu(self.fc3(x))
    x = F.elu(self.fc4(x))
    x = self.fc5(x) # Removed ELU activation and moved to the output layer
    # Removed x = self.softmax(x)
    return x

In [12]:
model = Net()

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [14]:
def train(model, dataloader_train, criterion, optimizer, epochs):
  model.train()
  for epoch in range(epochs):
    for features , labels in dataloader_train:
      optimizer.zero_grad()
      output = model(features)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

In [15]:
def test(model, dataloader_test, criterion):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for features, labels in dataloader_test:
      output = model(features)
      #total += criterion(output, labels).item()
      pred = output.argmax(dim=1)
      total += labels.size(0)
      correct += (output.argmax(dim=1) == labels).sum().item()
  accuracy = (correct / total) * 100 # Convert to percentage
  return accuracy

In [16]:
num_epoch = 150
train(model, dataloader_train, criterion, optimizer, num_epoch)

Epoch 1/150, Loss: 1.1938589811325073
Epoch 2/150, Loss: 1.6835688352584839
Epoch 3/150, Loss: 1.3795595169067383
Epoch 4/150, Loss: 1.6296440362930298
Epoch 5/150, Loss: 1.4694089889526367
Epoch 6/150, Loss: 1.0754302740097046
Epoch 7/150, Loss: 1.0672402381896973
Epoch 8/150, Loss: 1.462502121925354
Epoch 9/150, Loss: 0.8524600267410278
Epoch 10/150, Loss: 1.3686193227767944
Epoch 11/150, Loss: 0.8586801886558533
Epoch 12/150, Loss: 1.310300588607788
Epoch 13/150, Loss: 0.6529605388641357
Epoch 14/150, Loss: 0.979171872138977
Epoch 15/150, Loss: 0.7287566661834717
Epoch 16/150, Loss: 0.5455722808837891
Epoch 17/150, Loss: 0.6849369406700134
Epoch 18/150, Loss: 0.5825262665748596
Epoch 19/150, Loss: 1.0687988996505737
Epoch 20/150, Loss: 1.4606971740722656
Epoch 21/150, Loss: 0.4502708613872528
Epoch 22/150, Loss: 1.2727830410003662
Epoch 23/150, Loss: 1.0218617916107178
Epoch 24/150, Loss: 0.6880232095718384
Epoch 25/150, Loss: 1.11202073097229
Epoch 26/150, Loss: 0.958993136882782
E

In [17]:
test(model, dataloader_test, criterion)

72.39999999999999